## Install prerequisites

In [1]:
import time
import humanize
start_time = time.time()

In [2]:
# install Open Graph Benchmark
! pip install ogb

# install PyTorch Geometric
!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-geometric

     |████████████████████████████████| 45 kB 982 kB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 12.3 MB 470 kB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 21.6 MB 5.0 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 18.2 MB 4.3 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 6.3 MB 3.0 MB/s 
     |████████████████████████████████| 153 kB 3.5 MB/s 
     |████████████████████████████████| 231 kB 8.4 MB/s 
     |████████████████████████████████| 2.1 MB 11.6 MB/s 
     |████████████████████████████████| 45 kB 2.6 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.5.0-py3-none-any.whl size=267915 sha256=f50bea8cf4f4e0b8bc6ee91166c31c5f48639315a0cdf0b966dfe4089e44d0c5
  Stored in directory: /root/.cache/pip/

## Import libraries

In [3]:
import torch
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

from torch_sparse import SparseTensor
from torch_scatter import scatter
from torch_geometric.nn.inits import glorot, zeros

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

## Logger

In [4]:
class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]

    def add_result(self, run, result):
        assert len(result) == 3
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            argmax = result[:, 1].argmax().item()
            print(f'Run {run + 1:02d}:')
            print(f'Highest Train: {result[:, 0].max():.2f}')
            print(f'Highest Valid: {result[:, 1].max():.2f}')
            print(f'  Final Train: {result[argmax, 0]:.2f}')
            print(f'   Final Test: {result[argmax, 2]:.2f}')
        else:
            result = 100 * torch.tensor(self.results)

            best_results = []
            for r in result:
                train1 = r[:, 0].max().item()
                valid = r[:, 1].max().item()
                train2 = r[r[:, 1].argmax(), 0].item()
                test = r[r[:, 1].argmax(), 2].item()
                best_results.append((train1, valid, train2, test))

            best_result = torch.tensor(best_results)

            print(f'All runs:')
            r = best_result[:, 0]
            print(f'Highest Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 1]
            print(f'Highest Valid: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 2]
            print(f'  Final Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 3]
            print(f'   Final Test: {r.mean():.2f} ± {r.std():.2f}')

## Config

In [5]:
class args:
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    log_steps = 1
    num_layers = 3
    skip_layers = 3
    hidden_channels = 256
    dropout = 0.05
    lr = 0.01
    epochs = 1000
    eval_steps = 10
    runs = 1

In [6]:
device = torch.device(args.device)

dataset = PygNodePropPredDataset(name='ogbn-proteins')
split_idx = dataset.get_idx_split()
data = dataset[0]

x = scatter(data.edge_attr, data.edge_index[0], dim=0,
            dim_size=data.num_nodes, reduce='mean').to(device)

y_true = data.y.to(device)
train_idx = split_idx['train'].to(device)

edge_index = data.edge_index.to(device)
adj = SparseTensor(row=edge_index[0], col=edge_index[1])

  0%|          | 0/216 [00:00<?, ?it/s]

Downloaded 0.21 GB: 100%|██████████| 216/216 [00:06<00:00, 33.94it/s]


Extracting dataset/proteinfunc.zip
Processing...
Loading necessary files...
This might take a while.


  0%|          | 0/1 [00:00<?, ?it/s]

Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 442.86it/s]


Converting graphs into PyG objects...
Saving...
Done!


In [7]:
print(humanize.naturaldelta(start_time - time.time()))

3 minutes


## Model Definition

In [8]:
class GCNConv(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels

        self.weight = Parameter(torch.Tensor(in_channels, out_channels))
        self.bias = Parameter(torch.Tensor(out_channels))

        self.reset_parameters()

    def reset_parameters(self):
        glorot(self.weight)
        zeros(self.bias)
        

    def forward(self, x, adj):
        return adj @ x @ self.weight

In [9]:
class SAGEConv(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SAGEConv, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels

        self.weight = Parameter(torch.Tensor(in_channels, out_channels))
        self.root_weight = Parameter(torch.Tensor(in_channels, out_channels))
        self.bias = Parameter(torch.Tensor(out_channels))

        self.reset_parameters()

    def reset_parameters(self):
        glorot(self.weight)
        glorot(self.root_weight)
        zeros(self.bias)

    def forward(self, x, adj):
        out = adj.matmul(x, reduce="mean") @ self.weight
        out = out + x @ self.root_weight + self.bias
        return out

In [10]:
class ResidualGC(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, skip_n_channels,dropout):
        super(ResidualGC, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels))
        for _ in range(skip_n_channels):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
        
        self.compress = SAGEConv(hidden_channels + in_channels, hidden_channels)
        
        self.convs.append(GCNConv(hidden_channels, out_channels))
        
        self.dropout = dropout
    
    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        self.compress.reset_parameters()
            
    def forward(self, x, adj):
        x_i = x #identity
        for conv in self.convs[:-1]:
            x = conv(x, adj)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)

        x = torch.cat([x_i,x],1)
        
        x = self.compress(x, adj)
        x = self.convs[-1](x, adj)
        return x
    

In [11]:
class RGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(RGCN, self).__init__()
        self.init_conv = GCNConv(in_channels, hidden_channels)
        self.res1 = ResidualGC(hidden_channels,hidden_channels, hidden_channels,3, dropout)    
        self.bridge = GCNConv(hidden_channels, hidden_channels)     
        self.res2 = ResidualGC(hidden_channels,hidden_channels, out_channels,3, dropout)
        self.final_conv = GCNConv(out_channels, out_channels)    
        self.dropout = dropout

    def reset_parameters(self):
        self.init_conv.reset_parameters()
        self.res1.reset_parameters()
        self.bridge.reset_parameters()
        self.res2.reset_parameters()
        self.final_conv.reset_parameters()

    def forward(self, x, adj):
        x = self.init_conv(x,adj)
        x = self.res1(x,adj)
        x = self.bridge(x,adj)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.res2(x,adj)
        x = self.final_conv(x,adj)

        return x

In [12]:
class RRGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(RRGCN, self).__init__()
        self.init_conv = GCNConv(in_channels, hidden_channels)
        self.res1 = ResidualGC(hidden_channels,hidden_channels, hidden_channels,3, dropout)    
        self.bridge = SAGEConv(hidden_channels, hidden_channels)     
        self.res2 = ResidualGC(hidden_channels,hidden_channels, hidden_channels,3, dropout)
        self.compress = SAGEConv(hidden_channels*2, hidden_channels)
        self.final_conv = GCNConv(hidden_channels, out_channels)    
        self.dropout = dropout

    def reset_parameters(self):
        self.init_conv.reset_parameters()
        self.res1.reset_parameters()
        self.bridge.reset_parameters()
        self.res2.reset_parameters()
        self.final_conv.reset_parameters()
        self.compress.reset_parameters()

    def forward(self, x, adj):
        x = self.init_conv(x,adj)
        x_i = x
        x = self.res1(x,adj)
        x = self.bridge(x,adj)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.res2(x,adj)
        x = torch.cat([x,x_i],1) 
        x = self.compress(x,adj)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.final_conv(x,adj)

        return x

In [13]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj):
        for conv in self.convs[:-1]:
            x = conv(x, adj)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj)
        return x

In [14]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
        self.convs.append(GCNConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj):
        for conv in self.convs[:-1]:
            x = conv(x, adj)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj)
        return x

In [15]:
class MLP(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()

    def forward(self, x):
        for lin in self.lins[:-1]:
            x = lin(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return x

## Training and eval

In [16]:
def train(model, x, adj, y_true, train_idx, optimizer):
    model.train()
    criterion = torch.nn.BCEWithLogitsLoss()

    optimizer.zero_grad()
    out = model(x, adj)[train_idx]
    loss = criterion(out, y_true[train_idx].to(torch.float))
    loss.backward()
    optimizer.step()

    return loss.item()

In [17]:
@torch.no_grad()
def test(model, x, adj, y_true, split_idx, evaluator):
    model.eval()

    y_pred = model(x, adj)

    train_rocauc = evaluator.eval({
        'y_true': y_true[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['rocauc']
    valid_rocauc = evaluator.eval({
        'y_true': y_true[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['rocauc']
    test_rocauc = evaluator.eval({
        'y_true': y_true[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['rocauc']

    return train_rocauc, valid_rocauc, test_rocauc

In [18]:
models = []

models.append(GCN(x.size(-1), args.hidden_channels, 112, args.num_layers,args.dropout).to(device))

models.append(SAGE(x.size(-1), args.hidden_channels, 112, args.num_layers,args.dropout).to(device))


In [19]:
# Pre-compute GCN normalization.
adj = adj.set_diag()
deg = adj.sum(dim=1).to(torch.float)
deg_inv_sqrt = deg.pow(-0.5)
deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
adj = deg_inv_sqrt.view(-1, 1) * adj * deg_inv_sqrt.view(1, -1)
    
evaluator = Evaluator(name='ogbn-proteins')
logger = Logger(args.runs, args)
best_test_score = 0

for model in models:
    for run in range(args.runs):
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):

            loss = train(model, x, adj, y_true, train_idx, optimizer)

            if epoch % args.eval_steps == 0:
                result = test(model, x, adj, y_true, split_idx, evaluator)
                logger.add_result(run, result)

                if epoch % args.log_steps == 0:                
                    train_rocauc, valid_rocauc, test_rocauc = result
                    print(f'Run: {run + 1:02d}\t '
                          f'Epoch: {epoch:02d}\t '
                          f'Loss: {loss:.4f}\t '
                          f'Train: {100 * train_rocauc:.2f}%\t '
                          f'Valid: {100 * valid_rocauc:.2f}%\t'
                          f'Test: {100 * test_rocauc:.2f}%')
                    if(test_rocauc > best_test_score):
                        best_test_score = test_rocauc
                        save_path = "gcn.pth"
                        torch.save(model, save_path)
                        print("Model saved.")
    
        logger.print_statistics(run)
    logger.print_statistics()
    print("="*80)

Run: 01	 Epoch: 10	 Loss: 0.4876	 Train: 35.85%	 Valid: 34.56%	Test: 38.73%
Model saved.
Run: 01	 Epoch: 20	 Loss: 0.4325	 Train: 41.20%	 Valid: 33.54%	Test: 37.36%
Run: 01	 Epoch: 30	 Loss: 0.4127	 Train: 46.44%	 Valid: 34.44%	Test: 37.01%
Run: 01	 Epoch: 40	 Loss: 0.4091	 Train: 48.73%	 Valid: 35.40%	Test: 37.48%
Run: 01	 Epoch: 50	 Loss: 0.4038	 Train: 49.72%	 Valid: 36.79%	Test: 39.11%
Model saved.
Run: 01	 Epoch: 60	 Loss: 0.3998	 Train: 50.48%	 Valid: 37.43%	Test: 39.41%
Model saved.
Run: 01	 Epoch: 70	 Loss: 0.3955	 Train: 51.02%	 Valid: 38.70%	Test: 40.47%
Model saved.
Run: 01	 Epoch: 80	 Loss: 0.3911	 Train: 52.18%	 Valid: 41.12%	Test: 42.03%
Model saved.
Run: 01	 Epoch: 90	 Loss: 0.3867	 Train: 53.35%	 Valid: 43.85%	Test: 43.78%
Model saved.
Run: 01	 Epoch: 100	 Loss: 0.3821	 Train: 54.92%	 Valid: 47.06%	Test: 45.81%
Model saved.
Run: 01	 Epoch: 110	 Loss: 0.3774	 Train: 56.55%	 Valid: 49.58%	Test: 47.42%
Model saved.
Run: 01	 Epoch: 120	 Loss: 0.3733	 Train: 58.40%	 Valid: 5

### MLP

In [20]:
def train(model, x, y_true, train_idx, optimizer):
    model.train()
    criterion = torch.nn.BCEWithLogitsLoss()

    optimizer.zero_grad()
    out = model(x)[train_idx]
    loss = criterion(out, y_true[train_idx].to(torch.float))
    loss.backward()
    optimizer.step()

    return loss.item()

@torch.no_grad()
def test(model, x, y_true, split_idx, evaluator):
    model.eval()

    y_pred = model(x)

    train_rocauc = evaluator.eval({
        'y_true': y_true[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['rocauc']
    valid_rocauc = evaluator.eval({
        'y_true': y_true[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['rocauc']
    test_rocauc = evaluator.eval({
        'y_true': y_true[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['rocauc']

    return train_rocauc, valid_rocauc, test_rocauc


model = MLP(x.size(-1), args.hidden_channels, 112, args.num_layers,
                args.dropout).to(device)

evaluator = Evaluator(name='ogbn-proteins')
logger = Logger(args.runs, args)

for run in range(args.runs):
    model.reset_parameters()
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    for epoch in range(1, 1 + args.epochs):
        loss = train(model, x, y_true, train_idx, optimizer)

        if epoch % args.eval_steps == 0:
            result = test(model, x, y_true, split_idx, evaluator)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_rocauc, valid_rocauc, test_rocauc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_rocauc:.2f}%, '
                      f'Valid: {100 * valid_rocauc:.2f}% '
                      f'Test: {100 * test_rocauc:.2f}%')

    logger.print_statistics(run)
logger.print_statistics()

Run: 01, Epoch: 10, Loss: 0.3713, Train: 65.53%, Valid: 61.15% Test: 55.08%
Run: 01, Epoch: 20, Loss: 0.3426, Train: 66.51%, Valid: 60.74% Test: 57.72%
Run: 01, Epoch: 30, Loss: 0.3337, Train: 67.72%, Valid: 64.02% Test: 59.50%
Run: 01, Epoch: 40, Loss: 0.3268, Train: 69.17%, Valid: 66.50% Test: 62.55%
Run: 01, Epoch: 50, Loss: 0.3224, Train: 70.27%, Valid: 67.63% Test: 63.56%
Run: 01, Epoch: 60, Loss: 0.3184, Train: 71.51%, Valid: 68.67% Test: 64.29%
Run: 01, Epoch: 70, Loss: 0.3147, Train: 73.06%, Valid: 70.08% Test: 65.53%
Run: 01, Epoch: 80, Loss: 0.3109, Train: 74.49%, Valid: 71.25% Test: 67.18%
Run: 01, Epoch: 90, Loss: 0.3074, Train: 75.55%, Valid: 72.16% Test: 68.06%
Run: 01, Epoch: 100, Loss: 0.3047, Train: 76.33%, Valid: 72.87% Test: 68.47%
Run: 01, Epoch: 110, Loss: 0.3024, Train: 76.96%, Valid: 73.54% Test: 68.76%
Run: 01, Epoch: 120, Loss: 0.3007, Train: 77.37%, Valid: 73.78% Test: 68.70%
Run: 01, Epoch: 130, Loss: 0.2991, Train: 77.74%, Valid: 74.04% Test: 69.03%
Run: 01,

In [21]:
print(humanize.naturaldelta(start_time - time.time()))

40 minutes
